# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import (f1_score, precision_score, recall_score,
                             confusion_matrix, classification_report)
from sklearn.model_selection import (GridSearchCV, train_test_split,
                                     RandomizedSearchCV)
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier


In [ ]:
# load data from database

# Create an SQLAlchemy engine
engine = create_engine('sqlite:///../data/InsertDatabaseName.db')

# Load data using a SQL query
query = "SELECT * FROM clean_table"
df = pd.read_sql(query, engine)

X = df['message'].values
Y = df.drop(columns=['message', 'original', 'genre']).values

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    """ Function to tokenize and lemmatize text.
    Args:
        text (str): Input text to be tokenized.

    Returns:
        clean_tokens (list): List of cleaned tokens.
    """

    # convert all into lower case
    # Normaize data - remove dashes, puctuation etc.
    pun_regex = r"[^a-zA-Z0-9]"
    #text = text.apply(lambda x: re.sub(pun_regex, " ", x))
    text = re.sub(pun_regex, " ", text)
    # Remove all urls users may have left
    url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
                 '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    #text = text.apply(lambda x: re.sub(url_regex, '', x))
    text = re.sub(url_regex, " ", text)

    #tokens = text.apply(lambda x: word_tokenize(x))
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(LogisticRegression()))
        ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# train classifier
pipeline.fit(X_train, y_train)


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:

def report_summary(y_test, show_plot=False):
     """ Function to evaluate the model's performance.
    Args:
        y_test (numpy.ndarray): Test data labels.
        show_plot (bool): Flag to display confusion matrix heatmaps.
    """
        
    y_pred = pipeline.predict(X_test)
    for i in range(y_test.shape[1]):
        feature_name = df.drop(
            columns=["message", "original", "genre"]).columns[i]

        labels = np.unique(y_pred[:, 1])
        cf_matrix = confusion_matrix(
            y_test[:, i], y_pred[:, i], labels=labels)
        accuracy = (y_pred[:, i] == y_test[:, i]).mean()


        # Creating a report for precision, recall and f1 score
        print('\033[91m' + f'{feature_name}' + '\033[90m')
        print(classification_report(y_test[:,i], y_pred[:,i], labels=labels))
        if show_plot:
            # Creating a seaborn confusion Matrix
            plt.figure(figsize=(4, 3))  
            # Create the heatmap
            sns.heatmap(
                cf_matrix, annot=True,
                fmt='', cmap='Blues',
                xticklabels=labels,
                yticklabels=labels)
            # Add a title for each heatmap (optional)
            plt.title(f'Heatmap {feature_name}')
            plt.show()
        else:
            pass
        print("Accuracy:", accuracy)
        print()


report_summary(y_test, show_plot=False)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {

    #removing very common words (e.g., "the", "and") may be useful
    'vect__max_df': [0.75, 1.0],
    # Chose not to optimise by rare terms
    # Chose not to alter by reducing the count of tokens
    # Optimise regularisation strength for overfitting
    'clf__estimator__C': [0.01, 0.1, 1.0, 10.0],
    # Different can affect convergence speed and model performance
    'clf__estimator__solver': ['liblinear', 'saga'],
    'clf__estimator__max_iter': [200, 300]
}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

In [ ]:
# Best parameters
best_params = cv.best_params_
print("Best parameters found: ", best_params)

In [ ]:
# Best score
best_score = cv.best_score_
print("Best cross-validation score: ", best_score)

# Best estimator
best_estimator = cv.best_estimator_
print("Best estimator: ", best_estimator)

# Results of all grid search
results = cv.cv_results_
print("Grid search results: ", results)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
report_summary(y_test, show_plot=False)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import SelectKBest, chi2

# Define the pipeline
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(
                tokenizer=tokenize, stop_words='english', ngram_range=(1, 2))),
            ('tfidf', TfidfTransformer()),
            ('best', SelectKBest(chi2, k=100))  # Select top 100 features
        ])),
        ('pca', TruncatedSVD(n_components=100))
        # Dimensionality reduction with PCA
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

# Set hyperparameters for grid search
parameters = {
    'features__text_pipeline__vect__max_df': [0.75, 1.0],
    'features__text_pipeline__vect__ngram_range': [(1, 1), (1, 2)],
    'features__text_pipeline__tfidf__use_idf': [True, False],
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__min_samples_split': [2, 4]
}

# Define grid search
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, verbose=3, n_jobs=-1)

# Fit the model
cv.fit(X_train, y_train)

# Get the best model
best_model = cv.best_estimator_

# Evaluate the best model
report_summary(y_test, show_plot=False)

In [ ]:
# Best parameters
best_params = cv.best_params_
print("Best parameters found: ", best_params)

In [ ]:
pipeline.set_params(**best_params).fit(X_train, y_train)
report_summary(y_test, show_plot=True)

### 9. Export your model as a pickle file

In [ ]:
import pickle
# Save the model to a file
with open('classifier_md2.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.